### Simple Residual model in Keras

This notebook is simply for testing a resnet-50 inspired model built in Keras on a numerical signs dataset.

In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,ZeroPadding1D, Conv1D, Add
from keras.layers import MaxPooling2D, Dropout, AveragePooling2D
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
# Using a signs dataset, with images of numerical signs from 0-9
X = np.load("../data/sign-digits/X.npy")
y = np.load("../data/sign-digits/y.npy")
X.shape = (2062, 64, 64, 1)
X = shuffle(X,random_state=0)
y = shuffle(y,random_state=0)
print(X.shape)
print(y.shape)

(2062, 64, 64, 1)
(2062, 10)


In [3]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1)
print(X_train.shape)
print(X_test.shape)

(1855, 64, 64, 1)
(207, 64, 64, 1)


In [4]:
# Block corresponding with no change in size
def identity(X, f, filters):
    """
    filters: filters for each of the conv2D
    f: size of filter to use in mid block
    """
    F1,F2,F3 = filters
    
    X_earlier = X
    # Block 1
    X = Conv2D(F1, kernel_size=(1,1), strides=(1,1),padding="valid",kernel_initializer=keras.initializers.glorot_normal())(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)
    # Block 2
    X = Conv2D(F2, kernel_size=(f,f), strides=(1,1),padding="same",kernel_initializer=keras.initializers.glorot_normal())(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)
    # Block 3
    X = Conv2D(F3, kernel_size=(1,1), strides=(1,1),padding="valid",kernel_initializer=keras.initializers.glorot_normal())(X)
    X = BatchNormalization(axis=3)(X)
    X = Add()([X,X_earlier]) # Add earlier activation
    X = Activation("relu")(X)
    return X

In [5]:
# Block corresponding with a change in size
def conv_resid(X, f, filters,s):
    """
    filters: filters for each of the conv2D
    s: stride size to resize the output
    """
    F1,F2,F3 = filters
    X_earlier = X
    # Block 1
    X = Conv2D(F1, kernel_size=(1,1), strides=(s,s),padding="valid",kernel_initializer=keras.initializers.glorot_normal())(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)
    # Block 2
    X = Conv2D(F2, kernel_size=(f,f), strides=(1,1),padding="same",kernel_initializer=keras.initializers.glorot_normal())(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)
    # Block 3
    X = Conv2D(F3, kernel_size=(1,1), strides=(1,1),padding="valid",kernel_initializer=keras.initializers.glorot_normal())(X)
    X = BatchNormalization(axis=3)(X)
    # Resize earlier activation (X_earlier)
    X_earlier = Conv2D(F3, kernel_size=(1,1), strides=(s,s),padding="valid",kernel_initializer=keras.initializers.glorot_normal())(X_earlier)
    X_earlier = BatchNormalization(axis=3)(X_earlier)
    # Add earlier activation
    X = Add()([X,X_earlier])
    X = Activation("relu")(X)
    return X

In [6]:
# The Input shape for this model will be 64x64x1
def model(input_shape):
    X_input = Input(input_shape)
    X = ZeroPadding2D(padding=(3,3))(X_input)
    X = Conv2D(64,kernel_size=(7,7),padding="valid",kernel_initializer=keras.initializers.glorot_uniform())(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation(("relu"))(X)
    X = MaxPooling2D((3,3),strides=(2,2))(X)
    
    # indentity block 1
    X = conv_resid(X, 3, [64,64,256], 1)
    X = identity(X, 3, [64,64,256])
    X = identity(X, 3, [64,64,256])
    
    # Identity block 2
    X = conv_resid(X, 3, [128,128,512], 2)
    X = identity(X, 3, [128,128,512])
    X = identity(X, 3, [128,128,512])
    X = identity(X, 3, [128,128,512])
    
    # Identity block 3
    X = conv_resid(X, 3, [256, 256, 1024], 2)
    X = identity(X, 3, [256, 256, 1024])
    X = identity(X, 3, [256, 256, 1024])
    X = identity(X, 3, [256, 256, 1024])
    X = identity(X, 3, [256, 256, 1024])
    X = identity(X, 3, [256, 256, 1024])
    
    # Identity block 4
    X = conv_resid(X, 3, [512, 512, 2048], 2)
    X = identity(X, 3, [512, 512, 2048])
    X = identity(X, 3, [512, 512, 2048])
    
    X = AveragePooling2D((2,2), name="avg_pool")(X)
    # Flatten final layer
    X = Flatten()(X)
    X = Dense(10, activation="softmax",name="dense02",kernel_initializer = keras.initializers.glorot_normal())(X)
    
    model = Model(inputs=X_input, outputs=X, name="resnet")
    return model

In [7]:
resid_classi = model(X_train[0].shape)

In [8]:
resid_classi.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

In [9]:
resid_classi.fit(X_train, y_train,epochs=10,batch_size=10, validation_data=[X_test,y_test])

Train on 1855 samples, validate on 207 samples
Epoch 1/10
1855/1855 [==============================] - 756s 408ms/step - loss: 2.0275 - acc: 0.4965 - val_loss: 1.0932 - val_acc: 0.7874
Epoch 2/10
1855/1855 [==============================] - 697s 376ms/step - loss: 0.3569 - acc: 0.8852 - val_loss: 0.5941 - val_acc: 0.8261
Epoch 3/10
1855/1855 [==============================] - 701s 378ms/step - loss: 0.1776 - acc: 0.9488 - val_loss: 0.4580 - val_acc: 0.8792
Epoch 4/10
1855/1855 [==============================] - 690s 372ms/step - loss: 0.0929 - acc: 0.9725 - val_loss: 0.4911 - val_acc: 0.8647
Epoch 5/10
1855/1855 [==============================] - 688s 371ms/step - loss: 0.1306 - acc: 0.9601 - val_loss: 0.9518 - val_acc: 0.8019
Epoch 6/10
1855/1855 [==============================] - 691s 372ms/step - loss: 0.1014 - acc: 0.9698 - val_loss: 0.5530 - val_acc: 0.8261
Epoch 7/10
1855/1855 [==============================] - 688s 371ms/step - loss: 0.0902 - acc: 0.9725 - val_loss: 0.0832 - val